# Decision Tree Builder Model Using ID3 Algorithm and Testing on Iris Dataset
The code in a more structured format is publicly available (after the homework deadline) at:

https://github.com/smhash78/decision-tree-implementation/

# Initialization

In [53]:
from math import log2
from typing import Union, Dict, TypeVar, Tuple, List, Any
from time import time

import pandas as pd
from sklearn.model_selection import train_test_split

In [54]:
# CONSTANTS

# feature types
NOMINAL = 'NOM'
NUMERIC = 'NUM'

# methods
IG = 'INFORMATION_GAIN'
GR = 'GAIN_RATIO'

# evaluation metrics
ACCURACY = 'ACCURACY'
PRECISION = 'PRECISION'
RECALL = 'RECALL'
F1_SCORE = 'F1_SCORE'

# Implementation From Skratch

## Data class

In [55]:
T = TypeVar('T', bound='Data')

In [56]:
class Data:
    def __init__(
            self,
            X: pd.DataFrame,
            y: pd.Series,
            feature_types: Dict[str, str],
    ):
        self.X = X
        self.y = y
        self.feature_types = feature_types

    def __len__(self):
        return len(self.y)

    def get_feature_names(self):
        return self.X.columns.tolist()

    def get_dv_portions(
            self,
            xj: str,
            threshold: Union[int, float] = None
    ) -> Dict[str, T]:
        result = {}

        if self.feature_types[xj] == NOMINAL:
            for value in self.X[xj].unique():
                X_subset = self.X[self.X[xj] == value]
                y_subset = self.y[X_subset.index]

                result[value] = Data(X_subset, y_subset, self.feature_types.copy())

        elif self.feature_types[xj] == NUMERIC:
            if threshold is not None:
                X_subset_above = self.X[self.X[xj] >= threshold]
                y_subset_above = self.y[X_subset_above.index]

                X_subset_below = self.X[self.X[xj] < threshold]
                y_subset_below = self.y[X_subset_below.index]

                result = {
                    'above': Data(
                        X_subset_above,
                        y_subset_above,
                        self.feature_types,
                    ),
                    'below': Data(
                        X_subset_below,
                        y_subset_below,
                        self.feature_types,
                    ),
                }
            else:
                raise ValueError("The value of threshold can't be None when the feature is numeric.")

        return result

    def remove_feature(self, feature_name: str):
        self.X = self.X.drop(feature_name, axis=1)
        del self.feature_types[feature_name]


## Node classes

In [57]:
class Node:
    def __init__(
            self,
            selected_feature: str,
            feature_type: str,
            feature_values: Union[List[Any], None] = None,
            threshold: float = None,
            children: Union[Dict[Any, 'Node'], None] = None,
    ):
        self.selected_feature = selected_feature
        self.feature_type = feature_type

        if feature_values is None:
            self.feature_values = []
        else:
            self.feature_values = feature_values

        self.threshold = threshold

        if children is None:
            self.children = {}
        else:
            self.children = children

    def print_node(self, layer: int = 0, last_feature_value: Any = 'root'):
        indentation = '\t' * layer
        # nominal/categorical
        if self.threshold is None:
            print(f"{indentation}#({last_feature_value} -> {self.selected_feature}: {self.feature_values})#")
        # numeric
        else:
            print(f"{indentation}#({last_feature_value} -> {self.selected_feature}: {self.threshold})#")

        for key, child in self.children.items():
            child.print_node(layer + 1, key)

    def run_for_point(self, data_point: pd.Series) -> 'Node':
        if self.feature_type == NOMINAL:
            return self.children[data_point[self.selected_feature]]

        elif self.feature_type == NUMERIC:
            feature_value = data_point[self.selected_feature]

            if feature_value >= self.threshold:
                return self.children['above']

            else:
                return self.children['below']

In [58]:
class LeafNode:
    def __init__(
            self,
            label: Any = None,
    ):
        self.label = label

    def print_node(self, layer: int, last_feature_value: Any = 'root'):
        indentation = '\t' * layer
        print(f"{indentation}#({last_feature_value} -> label: {self.label})#")

## Functions

In [59]:
# basic functions
def calculate_log2(num: float):
    if num == 0:
        return inf
    return log2(num)


def calculate_entropy(
        data: Data,
) -> float:
    result = 0.0
    for label in data.y.unique():
        p = data.y.value_counts()[label] / len(data.y)
        result -= p * calculate_log2(p)

    return result


def calculate_conditional_entropy(
        data: Data,
        xj: str,
        threshold: Union[int, float, None] = None,
) -> float:
    result = 0.0
    for _, dv in data.get_dv_portions(xj, threshold).items():
        result += len(dv) / len(data) * calculate_entropy(dv)

    return result


def calculate_information_gain(
        data: Data,
        xj: str,
        threshold: Union[int, float, None] = None,
) -> float:
    return calculate_entropy(data) \
           - calculate_conditional_entropy(data, xj, threshold)


def calculate_split_info(
        data: Data,
        xj: str,
        threshold: Union[int, float, None] = None,
) -> float:
    result = 0.0
    for _, dv in data.get_dv_portions(xj, threshold).items():
        ratio = len(dv) / len(data)
        result -= ratio * calculate_log2(ratio)

    return result


def calculate_gain_ratio(
        data: Data,
        xj: str,
        threshold: Union[int, float, None] = None,
) -> float:
    return calculate_information_gain(data, xj, threshold) \
           / calculate_split_info(data, xj, threshold)


def calculate_gain(
        data: Data,
        xj: str,
        threshold: Union[int, float, None] = None,
        method: str = IG,
) -> float:
    if method == IG:
        return calculate_information_gain(data, xj, threshold)

    elif method == GR:
        return calculate_gain_ratio(data, xj, threshold)

In [60]:
# choosing functions
def find_best_threshold_gain(
        data: Data,
        xj: str,
        method: str = IG,
) -> Tuple[float, float]:
    unique_values = data.X[xj].unique()
    if len(unique_values) == 1:
        thresholds = unique_values
    else:
        thresholds = [
            (unique_values[i] + unique_values[i + 1]) / 2
            for i in range(len(unique_values) - 1)
        ]

    best_gain = -1
    best_threshold = None

    for threshold in thresholds:
        gain = calculate_gain(data, xj, threshold, method)
        if gain > best_gain:
            best_gain = gain
            best_threshold = threshold

    return best_threshold, best_gain


def find_best_gain(
        data: Data,
        xj: str,
        method: str = IG,
) -> Tuple[float, Union[int, float, None]]:
    if data.feature_types[xj] == NOMINAL:
        return calculate_gain(data, xj, method=method), None

    elif data.feature_types[xj] ==  NUMERIC:
        threshold, gain = find_best_threshold_gain(data, xj, method)

        return gain, threshold


def find_best_feature(
        data: Data,
        method: str =  IG,
) -> Tuple[Union[str, None], Union[int, float, None]]:
    best_gain = -1
    best_feature = None
    best_threshold = None

    for xj in data.get_feature_names():
        # The features that has only one value can never be the best feature
        if len(data.X[xj].unique()) == 1:
            continue
        gain, threshold = find_best_gain(data, xj, method)

        if gain > best_gain:
            best_gain = gain
            best_feature = xj
            best_threshold = threshold

    # when going deeper doesn't help
    # -1 happens when each feature has only one value
    if best_gain <= 0:
        return None, None

    return best_feature, best_threshold


In [61]:
# Tree construction
def construct_tree(
        data: Data,
        method: str = IG,
) -> Union[Node, LeafNode]:
    # all data sorted correctly
    if len(data.y.unique()) == 1:
        return LeafNode(data.y.iloc[0])

    # no feature left
    elif data.X.shape[1] == 0:
        return LeafNode(data.y.mode()[0])

    best_feature, threshold = find_best_feature(data, method)

    # none of the features is useful
    if best_feature is None:
        return LeafNode(data.y.mode()[0])

    feature_values = data.feature_types[best_feature] \
        if threshold is None \
        else None

    node = Node(
        selected_feature=best_feature,
        feature_type=data.feature_types[best_feature],
        feature_values=feature_values,
        threshold=threshold,
    )
    split_data = data.get_dv_portions(best_feature, threshold)

    for key, value in split_data.items():
        # nominal/categorical
        if threshold is None:
            value.remove_feature(best_feature)
        node.children[key] = construct_tree(value, method)

    return node

## ID3 Decision Tree Class

In [62]:
class DecisionTreeID3:
    def __init__(self):
        self.tree = None

    def train(
            self,
            data: Data,
            method: str = IG,
    ):
        self.tree = construct_tree(data, method)

    def test(
            self,
            test_data: Data,
            evaluation_metrics: Union[List[str], None] = None,
    ):
        if evaluation_metrics is None:
            evaluation_metrics = [ACCURACY]

        true_predictions = 0
        for i, row in test_data.X.iterrows():
            prediction = self.predict(row)
            if prediction == test_data.y.iloc[i]:
                true_predictions += 1

        results = {
            ACCURACY: true_predictions / len(test_data),
        }
        return {
            key: value
            for key, value in results.items()
            if key in evaluation_metrics
        }

    def predict(self, data_point: pd.Series):
        current_node = self.tree

        while not isinstance(current_node, LeafNode):
            current_node = current_node.run_for_point(data_point)

        return current_node.label

    def print_tree(self):
        self.tree.print_node(0)

# Train and Test

In [63]:
def train_and_print_test_results(
        train_data: Data,
        test_data: Data,
        sample_number: int,
        method: str = IG,
) -> None:
    print(f"Sample data {sample_number}, using {method}:")

    decision_tree = DecisionTreeID3()
    decision_tree.train(train_data, method)

    decision_tree.print_tree()
    print(f"\nAccuracy: {decision_tree.test(test_data)[ACCURACY]}\n")

In [64]:
# Sample test data 1
def run_test_data_1(method: str = IG):
    df = pd.DataFrame({
        'A': ['F', 'T', 'T', 'T'],
        'B': ['F', 'F', 'T', 'T'],
        'C': ['F', 'T', 'T', 'F'],
        'Y': ['F', 'T', 'F', 'T'],
    })

    X = df[['A', 'B', 'C']]
    y = df['Y']

    data = Data(
        X,
        y,
        {
            'A': NOMINAL,
            'B': NOMINAL,
            'C': NOMINAL,
        }
    )

    train_and_print_test_results(data, data, 1, method)

In [65]:
# Sample test data 2 (tennis game from slides)
def run_test_data_2(method: str = IG):
    dataset = [
        ['Sunny', 'Hot', 'High', 'Light', 'No'],
        ['Sunny', 'Hot', 'High', 'Strong', 'No'],
        ['Overcast', 'Hot', 'High', 'Light', 'Yes'],
        ['Rain', 'Mild', 'High', 'Light', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Light', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
        ['Sunny', 'Mild', 'High', 'Light', 'No'],
        ['Sunny', 'Cool', 'Normal', 'Light', 'Yes'],
        ['Rain', 'Mild', 'Normal', 'Light', 'Yes'],
        ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
        ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
        ['Overcast', 'Hot', 'Normal', 'Light', 'Yes'],
        ['Rain', 'Mild', 'High', 'Strong', 'No']
    ]

    df = pd.DataFrame(dataset, columns=['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Tennis?'])

    X = df[['Outlook', 'Temperature', 'Humidity', 'Wind']]
    y = df['Play Tennis?']

    data = Data(
        X,
        y,
        {
            'Outlook': NOMINAL,
            'Temperature': NOMINAL,
            'Humidity': NOMINAL,
            'Wind': NOMINAL,
        }
    )

    train_and_print_test_results(data, data, 2, method)

In [66]:
# Sample test data 3 (Iris)
def run_test_data_3(method: str = IG):
    column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    df = pd.read_csv('iris.data', header=None, names=column_names)

    X = df[column_names[:-1]]
    y = df[column_names[-1]]

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=42
    )
    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)

    train_data = Data(
        X_train,
        y_train,
        {key: NUMERIC for key in column_names[:-1]}
    )

    test_data = Data(
        X_test,
        y_test,
        {key: NUMERIC for key in column_names[:-1]}
    )

    train_and_print_test_results(train_data, test_data, 3, method)

In [67]:
def run_test_and_print(
        data_number: int,
        method: str = IG,
) -> None:
    now = time()
    if data_number == 1:
        run_test_data_1(method)
    elif data_number == 2:
        run_test_data_2(method)
    elif data_number == 3:
        run_test_data_3(method)
    print(f"It took {time() - now} seconds.")
    print()

## Test using information gain

In [68]:
run_test_and_print(1, IG)

Sample data 1, using INFORMATION_GAIN:
#(root -> A: NOM)#
	#(F -> label: F)#
	#(T -> B: NOM)#
		#(F -> label: T)#
		#(T -> C: NOM)#
			#(T -> label: F)#
			#(F -> label: T)#

Accuracy: 1.0

It took 0.039583683013916016 seconds.



In [69]:
run_test_and_print(2, IG)

Sample data 2, using INFORMATION_GAIN:
#(root -> Outlook: NOM)#
	#(Sunny -> Humidity: NOM)#
		#(High -> label: No)#
		#(Normal -> label: Yes)#
	#(Overcast -> label: Yes)#
	#(Rain -> Wind: NOM)#
		#(Light -> label: Yes)#
		#(Strong -> label: No)#

Accuracy: 1.0

It took 0.0554814338684082 seconds.



In [70]:
run_test_and_print(3, IG)

Sample data 3, using INFORMATION_GAIN:
#(root -> petal_length: 2.95)#
	#(above -> petal_length: 4.75)#
		#(above -> petal_length: 5.3)#
			#(above -> label: Iris-virginica)#
			#(below -> petal_width: 1.9)#
				#(above -> label: Iris-virginica)#
				#(below -> sepal_length: 6.35)#
					#(above -> label: Iris-versicolor)#
					#(below -> sepal_width: 3.1)#
						#(above -> label: Iris-versicolor)#
						#(below -> sepal_width: 2.75)#
							#(above -> label: Iris-virginica)#
							#(below -> sepal_width: 2.45)#
								#(above -> petal_width: 1.7000000000000002)#
									#(above -> label: Iris-virginica)#
									#(below -> label: Iris-versicolor)#
								#(below -> label: Iris-virginica)#
		#(below -> petal_width: 1.55)#
			#(above -> sepal_length: 5.45)#
				#(above -> label: Iris-versicolor)#
				#(below -> label: Iris-virginica)#
			#(below -> label: Iris-versicolor)#
	#(below -> label: Iris-setosa)#

Accuracy: 1.0

It took 1.5504844188690186 seconds.



## Test using gain ratio

In [71]:
run_test_and_print(1, GR)

Sample data 1, using GAIN_RATIO:
#(root -> A: NOM)#
	#(F -> label: F)#
	#(T -> B: NOM)#
		#(F -> label: T)#
		#(T -> C: NOM)#
			#(T -> label: F)#
			#(F -> label: T)#

Accuracy: 1.0

It took 0.04790019989013672 seconds.



In [72]:
run_test_and_print(2, GR)

Sample data 2, using GAIN_RATIO:
#(root -> Outlook: NOM)#
	#(Sunny -> Humidity: NOM)#
		#(High -> label: No)#
		#(Normal -> label: Yes)#
	#(Overcast -> label: Yes)#
	#(Rain -> Wind: NOM)#
		#(Light -> label: Yes)#
		#(Strong -> label: No)#

Accuracy: 1.0

It took 0.07091379165649414 seconds.



In [73]:
run_test_and_print(3, GR)

Sample data 3, using GAIN_RATIO:
#(root -> petal_length: 2.95)#
	#(above -> petal_width: 1.7000000000000002)#
		#(above -> petal_length: 4.85)#
			#(above -> label: Iris-virginica)#
			#(below -> sepal_length: 5.95)#
				#(above -> label: Iris-virginica)#
				#(below -> label: Iris-versicolor)#
		#(below -> petal_length: 5.25)#
			#(above -> label: Iris-virginica)#
			#(below -> petal_length: 4.75)#
				#(above -> sepal_width: 2.45)#
					#(above -> petal_length: 5.05)#
						#(above -> sepal_length: 6.15)#
							#(above -> label: Iris-virginica)#
							#(below -> label: Iris-versicolor)#
						#(below -> label: Iris-versicolor)#
					#(below -> label: Iris-virginica)#
				#(below -> petal_width: 1.55)#
					#(above -> sepal_length: 5.45)#
						#(above -> label: Iris-versicolor)#
						#(below -> label: Iris-virginica)#
					#(below -> label: Iris-versicolor)#
	#(below -> label: Iris-setosa)#

Accuracy: 1.0

It took 2.246432065963745 seconds.

